In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install vecstack


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from vecstack import stacking


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
trainData = pd.read_csv('/content/drive/MyDrive/Example/train.csv')
testData = pd.read_csv('/content/drive/MyDrive/Example/test.csv')

In [ ]:
a=list(trainData.count())
a

[1460,
 1460,
 1460,
 1201,
 1460,
 1460,
 91,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1452,
 1452,
 1460,
 1460,
 1460,
 1423,
 1423,
 1422,
 1423,
 1460,
 1422,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1459,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 770,
 1379,
 1379,
 1379,
 1460,
 1460,
 1379,
 1379,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 7,
 281,
 54,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460]

In [ ]:
s=len(a)
final=[]
for i in range(0,s):
  if a[i]<1000:
    final.append(i)
final

[6, 57, 72, 73, 74]

In [ ]:
for i in final:
  trainData.drop(trainData.columns[i], axis=1, inplace=True)
  testData.drop(testData.columns[i], axis=1, inplace=True)

In [ ]:
a=list(trainData.count())
a

[1460,
 1460,
 1460,
 1201,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1452,
 1452,
 1460,
 1460,
 1460,
 1423,
 1423,
 1422,
 1423,
 1460,
 1422,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1459,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 770,
 1379,
 1379,
 1460,
 1460,
 1379,
 1379,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 1460,
 7,
 281,
 1460,
 1460,
 1460,
 1460]

In [ ]:
trainCols = list(trainData.columns.values)
testCols = list(testData.columns.values)

In [ ]:
trainData

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,YrSold,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,NaN,NaN,0,2008,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,NaN,NaN,0,2007,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,NaN,NaN,0,2008,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,NaN,NaN,0,2006,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,NaN,NaN,0,2008,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,NaN,NaN,0,2007,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,NaN,MnPrv,0,2010,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,NaN,GdPrv,2500,2010,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,NaN,NaN,0,2010,Normal,142125


In [ ]:
testData

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,YrSold,SaleCondition
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,120,0,NaN,MnPrv,0,2010,Normal
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,36,0,0,0,0,NaN,NaN,12500,2010,Normal
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,34,0,0,0,0,NaN,MnPrv,0,2010,Normal
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,...,36,0,0,0,0,NaN,NaN,0,2010,Normal
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,...,82,0,0,144,0,NaN,NaN,0,2010,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,NaN,NaN,0,2006,Normal
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,...,24,0,0,0,0,NaN,NaN,0,2006,Abnorml
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,NaN,NaN,0,2006,Abnorml
1457,2918,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,...,32,0,0,0,0,NaN,MnPrv,700,2006,Normal


In [ ]:
s=len(a)
final=[]
for i in range(0,s):
  if a[i]!=1460:
    final.append(i)

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
for i in final:
  imputer = imputer.fit(trainData.iloc[:, i:i+1])
  trainData.iloc[:, i:i+1] = imputer.transform(trainData.iloc[:, i:i+1])

In [ ]:
testCols = list(testData.columns.values)

In [ ]:
a=list(testData.count())

In [ ]:
s=len(a)
final=[]
for i in range(0,s):
  if a[i]<1000:
    final.append(i)

In [ ]:
for i in final:
  trainData.drop(trainData.columns[i], axis=1, inplace=True)
  testData.drop(testData.columns[i], axis=1, inplace=True)

In [ ]:
a=list(testData.count())

In [ ]:
s=len(a)
final=[]
for i in range(0,s):
  if a[i]!=1459:
    final.append(i)

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
for i in final:
  imputer = imputer.fit(testData.iloc[:, i:i+1])
  testData.iloc[:, i:i+1] = imputer.transform(testData.iloc[:, i:i+1])

In [ ]:
trainData

In [ ]:
testData

In [ ]:
Y_train_original = trainData[['SalePrice']].copy()

In [ ]:
trainData.drop('SalePrice',axis=1,inplace=True)

In [ ]:
trainCols = list(trainData.columns.values)
testCols = list(testData.columns.values)

In [ ]:
catColumnsTrain = trainData.select_dtypes(exclude=['int','float64']).columns
catColumnsTest = testData.select_dtypes(exclude=['int','float64']).columns

In [ ]:
data = pd.concat([trainData, testData], axis=0)

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
Xcat = pd.DataFrame(ohe.fit_transform(data[catColumnsTrain]),columns=ohe.get_feature_names(),index=data.index)
df_train_1 = pd.concat([data,Xcat],axis=1)
df_train_1.drop(labels=catColumnsTrain,axis=1,inplace=True)

In [ ]:
df_train_1.shape

(2919, 257)

In [ ]:
trainData = df_train_1.iloc[:1460,:]
testData = df_train_1.iloc[1460:,:]

In [ ]:
trainData.shape

(1460, 257)

In [ ]:
testData.shape

(1459, 257)

In [ ]:
trainData.to_csv('/content/drive/MyDrive/Example/newtrain.csv')
testData.to_csv('/content/drive/MyDrive/Example/newtest.csv')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(trainData, Y_train_original, test_size=0.2, random_state=10)

In [ ]:
regressor = SVR()
parameters = {'C': [0.1, 1, 10, 100, 1000],
              'kernel': ['rbf']}

In [ ]:
svr_random = RandomizedSearchCV(regressor,parameters, n_iter=10, cv=5)
svr_random.fit(X_train, Y_train)
grid_parm=svr_random.best_params_

In [ ]:
regressor = SVR(**grid_parm)
regressor.fit(X_train, Y_train)
SVM_Preds = regressor.predict(testData)
SVM_Preds

array([169189.24000441, 177791.57279525, 180725.90767744, ...,
       197153.81654645, 164279.4242697 , 165182.02996051])

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, SVM_Preds):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/SVR_Preds.csv")

In [ ]:
DT=DecisionTreeRegressor()
parameters={"splitter":["best","random"],
            'min_samples_split' : range(10,100,10),
            "max_depth" : [2,4,6,8],
           "min_samples_leaf":[1,2,3,4,5,],
           "max_features":["auto","sqrt"],
           "max_leaf_nodes":[5,10,15] }

In [ ]:
dt_random = RandomizedSearchCV(DT, parameters, n_iter=10, cv=5)
grid_results = dt_random.fit(X_train, Y_train)
grid_parm=dt_random.best_params_

In [ ]:
dt = DecisionTreeRegressor(**grid_parm)

dt.fit(X_train, Y_train)

DT_Preds = dt.predict(testData)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, DT_Preds):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/DT_Preds.csv")

In [ ]:
parameters={'n_estimators':[5,10,20],'learning_rate':[0.01,.1]}
gbr = GradientBoostingRegressor()

In [ ]:
gbr_random = RandomizedSearchCV(gbr, parameters, n_iter=10, cv=5)
gbr_random.fit(X_train, Y_train)
grid_parm=gbr_random.best_params_

In [ ]:
gbr = GradientBoostingRegressor(**grid_parm)

gbr.fit(X_train, Y_train)

GBR_Preds = gbr.predict(testData)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, GBR_Preds):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/GBR_Preds.csv")

In [ ]:
parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)],
              'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive'], 'max_iter' :[100, 150]}
mlp = MLPRegressor()

In [ ]:
mlp_random =RandomizedSearchCV(mlp,parameters, n_iter=10, cv=5)
mlp_random.fit(X_train, Y_train)
grid_parm=mlp_random.best_params_

In [ ]:
mlp = MLPRegressor(**grid_parm)

mlp.fit(X_train, Y_train)

MLP_Preds = mlp.predict(testData)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, MLP_Preds):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/MLP_Preds.csv")

In [ ]:
rf = RandomForestRegressor()
parameters={'max_leaf_nodes':[2, 3, 4, 5, 6, 7], 'min_samples_split': [5, 10, 20, 50],
'max_depth': [5,10,15,20], 'max_features':[3,4,5], 'n_estimators': [50, 100, 200]}

In [ ]:
rf_random =RandomizedSearchCV(rf,parameters, n_iter=10, cv=5)
rf_random.fit(X_train, Y_train)
grid_parm=rf_random.best_params_

In [ ]:
rf = RandomForestRegressor(**grid_parm)

rf.fit(X_train, Y_train)

RF_Preds = rf.predict(testData)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, RF_Preds):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/RF_Preds.csv")

In [ ]:
models = [ RandomForestRegressor(), DecisionTreeRegressor(),  GradientBoostingRegressor() ]

S_Train, S_Test = stacking(
                           models,

                           X_train, Y_train, testData,

                          regression=True,

                           mode='oof_pred_bag',

                           needs_proba=False,

                           save_dir=None,

                           metric='accuracy_score',

                           n_folds=4,

                           stratified=True,

                           shuffle=True,

                           random_state=0,

                           )

In [ ]:
print(S_Train.shape)
print(S_Test.shape)

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(S_Train, Y_train)

parameters={"splitter":["best","random"],
            'min_samples_split' : range(10,100,10),
            "max_depth" : [2,4,6,8],
           "min_samples_leaf":[1,2,3,4,5,],
           "max_features":["auto","sqrt"],
           "max_leaf_nodes":[5,10,15] }

In [ ]:
dt_random = RandomizedSearchCV(dt,parameters,n_iter=15,cv=5)
dt_random.fit(S_Train, Y_train)
grid_parm=dt_random.best_params_

In [ ]:
dt = DecisionTreeRegressor(**grid_parm)
dt.fit(S_Train,Y_train)
dt_predict = dt.predict(S_Test)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, dt_predict):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/DT_StackedPreds.csv")

In [ ]:
mlp = MLPRegressor()
mlp.fit(S_Train, Y_train)

parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)], 'activation':['tanh', 'relu'], 'max_iter' :[100, 150]}

In [ ]:
mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15,cv=5)
mlp_random.fit(S_Train, Y_train)
grid_parm=mlp_random.best_params_
print(grid_parm)

In [ ]:
mlp = MLPRegressor(**grid_parm)
mlp.fit(S_Train,Y_train)
mlp_predict = mlp.predict(S_Test)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, mlp_predict):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/MLP_StackedPreds.csv")

In [ ]:
gbr = GradientBoostingRegressor()
gbr.fit(S_Train, Y_train)

parameters={'n_estimators':[5,10,20],'learning_rate':[0.01,.1]}

In [ ]:
gbr_random = RandomizedSearchCV(gbr,parameters,n_iter=15,cv=5)
gbr_random.fit(S_Train, Y_train)
grid_parm=gbr_random.best_params_
print(grid_parm)

In [ ]:
gbr = GradientBoostingRegressor(**grid_parm)
gbr.fit(S_Train,Y_train)
gbr_predict = gbr.predict(S_Test)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, gbr_predict):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/GBR_StackedPreds.csv")

In [ ]:
svm = SVR()

parameters = {'C': [0.1, 1, 10, 100, 1000],
              'kernel': ['rbf']}

In [ ]:
svm_random = RandomizedSearchCV(svm,parameters,n_iter=15,cv=5)
svm_random.fit(S_Train, Y_train)
grid_parm=svm_random.best_params_
print(grid_parm)

In [ ]:
svm = SVR(**grid_parm)
svm.fit(S_Train,Y_train)
svm_predict = svm.predict(S_Test)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, svm_predict):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/SVR_StackedPreds.csv")

In [ ]:
rf = RandomForestRegressor()
parameters={'max_leaf_nodes':[2, 3, 4, 5, 6, 7], 'min_samples_split': [5, 10, 20, 50],
'max_depth': [5,10,15,20], 'max_features':[3,4,5], 'n_estimators': [50, 100, 200]}

In [ ]:
rf_random =RandomizedSearchCV(rf,parameters, n_iter=10, cv=5)
rf_random.fit(S_Train, Y_train)
grid_parm=rf_random.best_params_

In [ ]:
rf = RandomForestRegressor(**grid_parm)

rf.fit(S_Train, Y_train)

rf_predict = rf.predict(S_Test)

In [ ]:
Pred_clf = []

for vals in zip(testData.Id, rf_predict):
    Pred_clf.append(vals)

Pred_clf = pd.DataFrame(Pred_clf,columns=['Id','SalePrice'])

Pred_clf.to_csv("/content/drive/MyDrive/Example/RF_StackedPreds.csv")